<a href="https://colab.research.google.com/github/0530hwi/KaggleStudy/blob/main/Kprototyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kmodes

In [ ]:
import os
import pandas as pd 
import numpy as np 

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import unicodedata
import datetime
from pytz import timezone

from kmodes.kprototypes import KPrototypes # clustering method

import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
mpl.rcParams['font.size'] = 9.0

# plt.rcParams['font.family'] = 'AppleGothic'

plt.rc('font', family='NanumBarunGothic') 

# 1. Data | without Region


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/Shareddrives/Socar_QuickNap/DATA/ExtractedData'
data_path = path + '/4_1_arrange_with_region.csv'

In [ ]:
# region 있는 데이터 

data = pd.read_csv(data_path)
data.head()

,region,reservation_start_at,use_total_hr,start_weekday,start_isweekend,age_group,gender,car_model
0,부산광역시 강서구,2018-12-27 12:20:00+00:00,141.69,3,0,3,male,중형SUV
1,경기도 고양시 일산동구,2018-12-27 18:00:00+00:00,231.75,3,0,2,female,경형
2,경기도 김포시,2018-12-28 11:00:00+00:00,124.83,4,0,3,female,경형
3,경기도 화성시,2018-12-28 14:00:00+00:00,104.84,4,0,2,male,소형SUV
4,경기도 의정부시,2018-12-28 20:20:00+00:00,187.40,4,0,2,male,준중형


# 2. Kprototypes


In [ ]:
# Kprototype 진행을 위해 reservation_start_at을 index로 변환하고자 함 
data.set_index('reservation_start_at', inplace = True)
data.head()

,region,use_total_hr,start_weekday,start_isweekend,age_group,gender,car_model
reservation_start_at,,,,,,,
2018-12-27 12:20:00+00:00,부산광역시 강서구,141.69,3,0,3,male,중형SUV
2018-12-27 18:00:00+00:00,경기도 고양시 일산동구,231.75,3,0,2,female,경형
2018-12-28 11:00:00+00:00,경기도 김포시,124.83,4,0,3,female,경형
2018-12-28 14:00:00+00:00,경기도 화성시,104.84,4,0,2,male,소형SUV
2018-12-28 20:20:00+00:00,경기도 의정부시,187.40,4,0,2,male,준중형


In [ ]:
# region 한글 to ALPHABET CATEGORY
region_ = sorted(list(set(data['region'])))
print(len(region_))
alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r','s', 't', 'u','v', 'w', 'x', 'y']

r_set = {}
for r, a in zip(region_, alpha):
  r_set[r] = a


25


In [ ]:
r_set

{'경기도 고양시 덕양구': 'a',
 '경기도 고양시 일산동구': 'b',
 '경기도 고양시 일산서구': 'c',
 '경기도 광명시': 'd',
 '경기도 구리시': 'e',
 '경기도 김포시': 'f',
 '경기도 남양주시': 'g',
 '경기도 성남시 분당구': 'h',
 '경기도 성남시 수정구': 'i',
 '경기도 성남시 중원구': 'j',
 '경기도 안양시 동안구': 'k',
 '경기도 안양시 만안구': 'l',
 '경기도 양평군': 'm',
 '경기도 의정부시': 'n',
 '경기도 하남시': 'o',
 '경기도 화성시': 'p',
 '부산광역시 강서구': 'q',
 '세종특별자치시 세종시': 'r',
 '울산광역시 남구': 's',
 '울산광역시 동구': 't',
 '울산광역시 북구': 'u',
 '울산광역시 울주군': 'v',
 '울산광역시 중구': 'w',
 '전라북도 전주시 덕진구': 'x',
 '전라북도 전주시 완산구': 'y'}

In [ ]:
# car_model 한글 to ALPHABET CATEGORY
car_ = sorted(list(set(data['car_model'])))

car_set = {}
for car, a in zip(car_, alpha):
  car_set[car] = a

car_set

{'경형': 'a',
 '대형': 'b',
 '소형': 'c',
 '소형SUV': 'd',
 '수입': 'e',
 '승합': 'f',
 '준대형': 'g',
 '준중형': 'h',
 '준중형SUV': 'i',
 '중형': 'j',
 '중형SUV': 'k'}

In [ ]:
data['region']= data['region'].map(r_set)
data['car_model']= data['car_model'].map(car_set)
data.head()

,region,use_total_hr,start_weekday,start_isweekend,age_group,gender,car_model
reservation_start_at,,,,,,,
2018-12-27 12:20:00+00:00,q,141.69,3,0,3,male,k
2018-12-27 18:00:00+00:00,b,231.75,3,0,2,female,a
2018-12-28 11:00:00+00:00,f,124.83,4,0,3,female,a
2018-12-28 14:00:00+00:00,p,104.84,4,0,2,male,d
2018-12-28 20:20:00+00:00,n,187.40,4,0,2,male,h


In [ ]:
data_v = data.values
categorical_index = [0, 2, 3, 4, 5, 6]

## 2-1. Elbow Curve | Check # of clustering 
- start_weekday 있는 데이터로 진행 

In [ ]:
# Function for plotting elbow curve

def plot_elbow_curve(start, end, data):
    no_of_clusters = list(range(start, end+1))
    cost_values = []
    
    for k in no_of_clusters:
        test_model = KPrototypes(n_clusters=k, init='Huang', random_state=0)
        test_model.fit_predict(data, categorical=categorical_index)
        cost_values.append(test_model.cost_)
        
    sns.set_theme(style="whitegrid", palette="bright", font_scale=1.2)
    
    plt.figure(figsize=(15, 7))
    ax = sns.lineplot(x=no_of_clusters, y=cost_values, marker="o", dashes=False)
    ax.set_title('Elbow curve', fontsize=18)
    ax.set_xlabel('No of clusters', fontsize=14)
    ax.set_ylabel('Cost', fontsize=14)
    ax.set(xlim=(start-0.1, end+0.1))
    plt.plot()


# Plotting elbow curve for k=2 to k=10
plot_elbow_curve(2,10,data_v)

In [ ]:
plt.savefig('/content/drive/Shareddrives/Socar_QuickNap/02ML/Kprotopytes_ElbowCurve')

# 2-2. # of Cluster Test

## 2-2-1. Test 1

In [ ]:
kproto_1 = KPrototypes(n_clusters=4, verbose=2, max_iter=20)
kproto_1.fit(data_v, categorical=categorical_index)

In [ ]:
clusters_1 = kproto_1.predict(data_v, categorical=categorical_index)

In [ ]:
data['cluster_1'] = list(clusters_1)

In [ ]:
data.head()
data.to_csv('/content/drive/Shareddrives/Socar_QuickNap/02ML/Kprotopytes_1.csv')

## 2-2-2. Test 2

In [ ]:
kproto_2 = KPrototypes(n_clusters=6, verbose=2, max_iter=20)
kproto_2.fit(data_v, categorical=categorical_index)

In [ ]:
clusters_2 = kproto_2.predict(data_v, categorical=categorical_index)

In [ ]:
data['cluster_2'] = list(clusters_2)

In [ ]:
data.to_csv('/content/drive/Shareddrives/Socar_QuickNap/02ML/Kprotopytes_2.csv')

## 2-2-3. Test 3

In [ ]:
kproto_3 = KPrototypes(n_clusters=10, verbose=2, max_iter=20)
kproto_3.fit(data_v, categorical=categorical_index)

In [ ]:
clusters_3 = kproto_3.predict(data_v, categorical=categorical_index)

In [ ]:
data['cluster_3'] = list(clusters_3)

In [ ]:
data.head()

In [ ]:
data.to_csv('/content/drive/Shareddrives/Socar_QuickNap/02ML/Kprotopytes_Fin.csv')

# 3. Cluster별 확인 

In [ ]:
# ANOVA 확인
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('use_total_hr_mm ~ C(cluster_id)', data).fit()
anova_lm(model)

In [ ]:
# # 사후분석을 위한 준비
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(data.use_total_hr, data.cluster_1)


In [ ]:
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf')
result[0]

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

hsd = pairwise_tukeyhsd(data['use_total_hr'], data['cluster_1'], alpha=0.05)
hsd.summary()

# 4. 예측 진행 

In [ ]:
# car_model onehot으로 분리시킨 데이터 들고와서 cluster 정보 취합하기 

## 4-1. VAR | Multivariable Time Series

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
# plt.style.use(['fivethirtyeight'])
# plt.style.use(['bmh'])

import os 
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller # stationary 확인 
from statsmodels.tsa.seasonal import seasonal_decompose # decomposition

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.graphics.tsaplots as sgt 

from statsmodels.tsa.arima_model import ARIMA
# from pmdarima.arima import auto_arima # ARIMA parameter auto 확인 
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math 

### 4-1-1. interpolate(method = 'time')

In [ ]:
data_m = data.interpolate(method = 'time')
data_m.isnull().sum()

In [ ]:
rcParams['figure.figsize'] = 10, 5
plt.plot(data_m['count'], color = 'tab:blue', label = 'count')
plt.legend(loc = 'best')
plt.show()

In [ ]:
def plot_rolling_data(data, window = 4*24*7): # 일주일 window 
    rollmean = data.rolling(window = window).mean()
    rollstd = data.rolling(window = window).std()
    print(rollstd.count())
    # 구간별 평균, 표준편차, original 값 확인하기 위해 plotting
    orig = plt.plot(data, color = 'tab:blue', label = 'orig')
    avg = plt.plot(rollmean, color = 'tab:green', label = 'avg')
    std = plt.plot(rollstd, color = 'tab:orange', label = 'std')
    plt.legend(loc = 'count')
    plt.title('Rolling Count_Orig, Avg, Std')
    plt.show()

In [ ]:
# ADF | 최소 lag
def augmented_dickey_fuller(data):
    adfuller_test = adfuller(data, autolag = 'AIC') # 최소 lag 찾음
    adfuller_test_output = pd.Series(adfuller_test[:4], 
                            index = ['test_statistic', 'pvalue', 'num_of_lags', 'num_of_observation'])
    for key, value in adfuller_test[4].items():
        adfuller_test_output['critical_values (%s)' %key] = value
    print(adfuller_test_output)

In [ ]:
# ADF | 최대 lag
def augmented_dickey_fuller_2(data):
    adfuller_test = adfuller(data, autolag = 't-stat') # 최대 lag 찾음
    adfuller_test_output = pd.Series(adfuller_test[:4], 
                            index = ['test_statistic', 'pvalue', 'num_of_lags', 'num_of_observation'])
    for key, value in adfuller_test[4].items():
        adfuller_test_output['critical_values (%s)' %key] = value
    print(adfuller_test_output)

pvalue 0.05 이상이면 stationary하다는 귀무가설 기각 불가, 즉 unstationary하다는 것 

In [ ]:
# np.log를 사용하여 데이터 로그화 진행 
ts_log = np.log(data)

# original data와 np.log화 시킨 데이터 plot
fig, ax1 = plt.subplots()
ax1.plot(ts_log, color = 'tab:blue', label = 'ts_log')

ax2 = ax1.twinx()
ax2.plot(ts, color = 'tab:green', label = 'ts')
plt.legend(loc = 'best')
plt.grid(False)
plt.show()


Decomposition

In [ ]:
# period = 4*24*7로 일주일 간격으로 진행함 
# 시간폭에 따라 변화가 존재하기 때문에 multiplicative로 진행 
decomposition = seasonal_decompose(ts_log, model = 'multiplicative', 
                                    period = 4*24*7)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid # original - trend - seasonal

plt.rcParams['figure.figsize'] = (10, 6)
plt.subplot(411)
plt.plot(ts_log, label = 'Original')
plt.legend(loc = 'best')
plt.subplot(412)
plt.plot(trend, label = 'Trend')
plt.legend(loc = 'best')
plt.subplot(413)
plt.plot(seasonal, label = 'Seasonality')
plt.legend(loc = 'best')
plt.subplot(414)
plt.plot(residual, label = 'Residuals')
plt.legend(loc = 'best')
plt.tight_layout()
plt.show()

Null값 처리 
- decomposition period만큼 앞에 데이터가 Null로 사라짐 

In [ ]:
# null 값 처리 
residual.dropna(inplace = True)
augmented_dickey_fuller(residual)

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5

plot_rolling_data(residual, window = 4*24*7) 

Train & Test 분리 
- validation set도 설정해야하나 

In [ ]:
train, test = ts_log[:int(len(ts_log)*0.9)], ts_log[int(len(ts_log)*0.9):]

### 4-1-2. interpolate(method = 'time') 없이 진행 

# 5. 